In [2]:
import tensorflow as tf #importing libraries 
import pandas as pd #data frame
import numpy as np #array
import matplotlib.pyplot as plt #visualization
import nltk #nlp lib
import re #lower case
from nltk.corpus import stopwords #unwanted words lib(stopwords)
from nltk.stem.porter import PorterStemmer #stemming
from sklearn.model_selection import train_test_split #dividing data sets train test 
from tensorflow.keras.layers import Dropout #droup out layer
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Embedding #embidding layer lib
from tensorflow.keras.preprocessing.sequence import pad_sequences #padding post 0-0-0
from tensorflow.keras.models import Sequential #sequential model lib
from tensorflow.keras.preprocessing.text import one_hot #one hot represnet 
from tensorflow.keras.layers import LSTM #lstm lib
from tensorflow.keras.layers import Dense 

In [3]:
datatrain=pd.read_csv("/content/drive/My Drive/Colab Notebooks/fakenewslstm/train.csv") #loading datasets 

In [4]:
datatrain.head(3)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [5]:
datatrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
datatrain.dropna(inplace=True) #droping nan values

In [7]:
datatrain.reset_index(inplace=True) #reseting index after droping

In [8]:
x=datatrain["title"] #dependent feature storing in X
print(x.shape)
x.head(3)

(18285,)


0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
Name: title, dtype: object

In [9]:
y=datatrain["label"] #independent feature
print(y.shape)
y.head(3)

(18285,)


0    1
1    0
2    1
Name: label, dtype: int64

In [10]:
nltk.download("stopwords") #loading stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
ps = PorterStemmer() #steming processes
corpus = []
for i in range(0, len(x)): #till the length of X (means text)
    review = re.sub('[^a-zA-Z]', ' ', x[i]) #special word apart from A-Z||a-z will be replaced by " " in text field
    review = review.lower() # lowering  letter
    review = review.split() #spliting each word
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #removing stop waords
    review = ' '.join(review)
    corpus.append(review)

In [12]:
voc_size=5000 # intializing dict size
onehot_repr=[one_hot(words,voc_size)for words in corpus] #performing one hot encoder after tokenizibg the text field to sentence->stemming ->stop word

In [13]:
sent_length=20 ## creating max length of each sentence
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length) #padding zero if length is less than 20

In [14]:
embedding_vector_features=40  # dimension of feature
model=Sequential() #model creation
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #adding embedding layer
model.add(LSTM(100)) #adding lstm layer
model.add(Dense(1,activation='sigmoid')) #output layer
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
xfinal=np.array(embedded_docs)
yfinal=np.array(y)
print(xfinal.shape, yfinal.shape)

(18285, 20) (18285,)


In [17]:
print(len(embedded_docs),y.shape)

18285 (18285,)


drop out layer added 


In [ ]:
"""embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])""

In [18]:
xtrain, xtest, ytrain, ytest = train_test_split(xfinal, yfinal, test_size=.20, random_state=42)

In [19]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=2,batch_size=64)

Epoch 1/2
229/229 [==============================] - 3s 15ms/step - loss: 0.3091 - accuracy: 0.8588 - val_loss: 0.1939 - val_accuracy: 0.9144
Epoch 2/2
229/229 [==============================] - 3s 13ms/step - loss: 0.1362 - accuracy: 0.9463 - val_loss: 0.2117 - val_accuracy: 0.9180


In [20]:
ypred=model.predict_classes(xtest)
print(ypred)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[1]
 [0]
 [0]
 ...
 [1]
 [0]
 [1]]


In [21]:
confusion_matrix(ytest,ypred)

array([[1875,  207],
       [  93, 1482]])

In [22]:
accuracy_score(ytest,ypred)

0.9179655455291222